In [1]:
import cv2
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt


In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


In [3]:
import cv2
import numpy as np
import mediapipe as mp
from filterpy.kalman import KalmanFilter

# Initialize MediaPipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize video capture
cap = cv2.VideoCapture("../test/30CST.mp4")
ret, old_frame = cap.read()
if not ret:
    print("Failed to load video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Setup for Optical Flow
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}

# Get FPS of the video
fps = cap.get(cv2.CAP_PROP_FPS)
time_interval = 1 / fps
print(f"Video FPS: {fps}")

# Function to initialize Kalman Filter
def initialize_kf():
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.F = np.array([[1]])    # state transition matrix
    kf.H = np.array([[1]])    # measurement function
    kf.P *= 1000.             # covariance matrix, large initial uncertainty
    kf.R = 0.1                # measurement noise
    kf.Q = 1                  # process noise
    return kf

# Initialize Kalman Filters for displacements in x and y directions
kf_x = initialize_kf()
kf_y = initialize_kf()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_image)
    annotated_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        keypoints = np.array([
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * frame.shape[1],
             landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * frame.shape[0]],
            [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * frame.shape[1],
             landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * frame.shape[0]],
            [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * frame.shape[1],
             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * frame.shape[0]]
        ], dtype=np.float32).reshape(-1, 1, 2)

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, keypoints, None, **lk_params)

        # Apply Kalman filter on displacement
        for i, (new, old) in enumerate(zip(p1, keypoints)):
            dx = new[0][0] - old[0][0]
            dy = new[0][1] - old[0][1]
            kf_x.predict()
            kf_y.predict()
            kf_x.update(dx)
            kf_y.update(dy)
            filtered_dx = kf_x.x[0]
            filtered_dy = kf_y.x[0]
            cv2.line(annotated_image, (int(old[0][0]), int(old[0][1])), (int(old[0][0] + filtered_dx), int(old[0][1] + filtered_dy)), (0, 255, 0), 2)
            cv2.circle(annotated_image, (int(new[0][0]), int(new[0][1])), 5, (0, 0, 255), -1)

        old_gray = frame_gray.copy()
        keypoints = p1.copy()

    cv2.imshow('Mediapipe Feed', annotated_image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'filterpy'

In [ ]:
# keypoints_history
# displacements
velocities

[3482.1524047851562,
 -1213.5193634033203,
 8780.846557617188,
 5619.309997558594,
 8150.855712890625,
 -24904.82666015625,
 591.7527008056641,
 -115.63138961791992,
 -266.38097763061523,
 -327.7308654785156,
 274.6036148071289,
 272.55924224853516,
 -324.9575901031494,
 -222.14439868927002,
 -77.18755960464478,
 125.80458641052246,
 -35.174546241760254,
 159.47426319122314,
 -354.4691562652588,
 49.24164533615112,
 232.57608890533447,
 -115.5732536315918,
 -84.61878776550293,
 118.04121494293213,
 -62.17456340789795,
 -82.17591762542725,
 323.8488006591797,
 -315.50225257873535,
 -17.725106477737427,
 324.4598579406738,
 -280.4121494293213,
 -68.20004940032959,
 397.2923183441162,
 -212.9305601119995,
 -133.74183654785156,
 188.43003273010254,
 -86.57763004302979,
 -113.19017887115479,
 26.28058433532715,
 -20.285897254943848,
 133.72604370117188,
 -21.180052757263184,
 -46.97061538696289,
 73.34638595581055,
 -85.39977550506592,
 -71.93525791168213,
 17.32344388961792,
 62.3112273216